In [ ]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
bank_marketing = fetch_ucirepo(id=222) 
  
# data (as pandas dataframes) 
X = bank_marketing.data.features 
y = bank_marketing.data.targets 

# metadata 
print(bank_marketing.metadata) 

# variable information 
print(bank_marketing.variables) 

In [ ]:
type(bank_marketing)

In [ ]:
bank_marketing.head()

In [ ]:
bank_marketing.shape